In [1]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import numpy as np
import pandas as pd

In [2]:
res = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1/1000/')
data = json.loads(res.content)
df_1 = pd.DataFrame.from_dict(data['rentBikeStatus']['row'])


res_2 = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1001/2000')
data_2 = json.loads(res_2.content)
df_2 = pd.DataFrame.from_dict(data_2['rentBikeStatus']['row'])


fin_data = pd.concat([df_1, df_2],ignore_index=True)

In [3]:
fin_data['station_No']=fin_data['stationName'].apply(lambda e: e.split('.')[0])
fin_data['station_Name']=fin_data['stationName'].apply(lambda e: e.split('.')[1])

In [4]:
fin_data = fin_data.drop(columns=['stationName'])

In [5]:
arr = pd.read_csv('using_No.csv',names = ["station_No"])

In [6]:
arr['station_No'] = arr['station_No'].astype(str)

In [7]:
fin_data = fin_data[fin_data["station_No"].isin(arr['station_No'])]
fin_data = fin_data.reset_index(drop=True)

In [8]:
dt_in = pd.read_csv("data_in.csv")
dt_out = pd.read_csv("data_out.csv")

In [9]:
data = pd.merge(dt_in, dt_out, on='station_id')

In [10]:
outcome_1=pd.merge(fin_data, data, how='outer', left_index=True, right_index=True)

In [11]:
outcome_1 = outcome_1.drop(columns=['station_id'])

In [12]:
dt = pd.read_csv('공공자전거 대여소 정보_201905.csv', engine='python')

In [13]:
dt= dt[dt["대여소번호"].isin(arr['station_No'])]
dt = dt.reset_index(drop=True)
dt = dt.drop(columns=['대여소명', '위도', '경도','거치대수'])

In [14]:
cols = dt.columns.tolist()

cols = cols[-1:] + cols[:-1]

cols

dt = dt[cols]

In [15]:
final_out = pd.concat([dt, outcome_1], axis=1)
final_out = final_out.drop(columns='station_No')
final_out = final_out.rename(columns={'대여소번호':'station_Num'})

In [16]:
station_flag = final_out['station_Name'].str.contains("역 ")
final_out['station_flag'] = station_flag

In [17]:
searchfor = ['학교', '대 ']
school_flag = final_out['station_Name'].str.contains('|'.join(searchfor))
final_out['school_flag'] = school_flag

In [18]:
final_out.to_csv('final_out.csv',encoding='utf-8-sig',index=False)